# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [7]:
from IPython.display import display

We will also be working on the following files/urls

In [15]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [16]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [17]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[10000:11000])

ection#Subsections" title="Help:Section">subheadings</a>.</span> <small><i>(January 2018)</i></small></div>
</td>
</tr>
</table>
<table class="vertical-navbox nowraplinks hlist" style="float:right;clear:right;width:22.0em;margin:0 0 1.0em 1.0em;background:#f9f9f9;border:1px solid #aaa;padding:0.2em;border-spacing:0.4em 0;text-align:center;line-height:1.4em;font-size:88%">
<tr>
<th class="navbox-title" style="padding:0.2em 0.4em 0.2em;font-size:145%;line-height:1.2em"><a href="/wiki/Sociology" title="Sociology">Sociology</a></th>
</tr>
<tr>
<td style="padding:0.2em 0 0.4em"><a href="/wiki/File:Social_Network_Diagram_(segment).svg" class="image"><img alt="Social Network Diagram (segment).svg" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Social_Network_Diagram_%28segment%29.svg/250px-Social_Network_Diagram_%28segment%29.svg.png" width="250" height="97" class="thumbborder" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Social_Network_Diagram_%28segment%29.svg/375px

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [18]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
(window.RLQ=window.RLQ||[]).push(functio


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [19]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [20]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)



Content analysis is a research method for studying documents and communication artifacts, which can be texts of various formats, pictures, audio or video. Social scientists use content analysis to quantify patterns in communication, in a replicable and systematic manner.[1] One of the key advantage of this research method is to analyse social phenomena in a non-invasive way, in contrast to simulating social experiences or collecting survey answers.
Practices and philosophies of content analysis vary between scholarly communities. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful patterns.[2][3] After labeling a large set of media, a researcher is able to statistically estimate the proportions of patterns in the texts, as well as correlations between patterns.


We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [21]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0                                                    
1   \nContent analysis is a research method for st...
2   Practices and philosophies of content analysis...
3   Computers are increasingly used in content ana...
4                                                    
5                                                    
6   Content analysis is best understood as a broad...
7   The simplest and most objective form of conten...
8   A further step in analysis is the distinction ...
9   More generally, content analysis is research u...
10  By having contents of communication available ...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Recently, Arash Heydarian Pashakhanlou has arg...
17  Content analysis can als

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [11]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [12]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [13]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [2]:
# President Obama's 8th state of the union address
obamaSpeechRequest = requests.get('https://en.wikisource.org/wiki/Barack_Obama%27s_Eighth_State_of_the_Union_Address')
print(obamaSpeechRequest.text[40000:41000])

ing approach to campaign finance reform can’t pass muster in the courts, we need to work together to find a real solution, 'cause it's a problem. And most of you don't like raising money. I know, I've done it. We've got to make it easier to vote, not harder. We need to modernize it for the way we live now. This is America, we want to make it easier for people to participate. And over the course of this year, I intend to travel the country to push for reforms that do just that.</p>
<p>But I can’t do these things on my own. Changes in our political process—in not just who gets elected but how they get elected—that will only happen when the American people demand it. It will depend on you. That’s what’s meant by a government of, by, and for the people.</p>
<p>What I’m asking for is hard. It’s easier to be cynical; to accept that change isn’t possible, and politics is hopeless, and to believe that our voices and actions don’t matter. But if we give up now, then we forsake a better future. 

In [3]:
obamaSpeechSoup = bs4.BeautifulSoup(obamaSpeechRequest.text, 'html.parser')
print(obamaSpeechSoup.text[:200])





Barack Obama's Eighth State of the Union Address - Wikisource, the free online library
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "


In [4]:
obama_speech_save = 'obama_speech_8.html'

with open(obama_speech_save, mode='w', encoding='utf-8') as f:
    f.write(obamaSpeechRequest.text)

In [5]:
contentPTags = obamaSpeechSoup.body.findAll('p')
for pTag in contentPTags[:5]:
    print(pTag.text)

Mr. Speaker, Mr. Vice President, Members of Congress, my fellow Americans:
Tonight marks the eighth year that I’ve come here to report on the State of the Union. And for this final one, I’m going to try to make it shorter. I know some of you are antsy to get back to Iowa. I've been there. I'll be shaking hands afterwards if you want some tips.
Now I understand that because it’s an election season, expectations for what we’ll achieve this year are low. But Mr. Speaker, I appreciate the constructive approach that you and the other leaders took at the end of last year to pass a budget and make tax cuts permanent for working families. So I hope we can work together this year on some bipartisan priorities like criminal justice reform, and helping people who are battling prescription drug abuse and heroin abuse. So who knows: we might surprise the cynics again.
But tonight, I want to go easy on the traditional list of proposals for the year ahead. Don’t worry: I’ve got plenty, from helping s

In [6]:
contentParagraphs = []
contentSentences = []
contentNumbers = []
contentWords = {}
for pTag in contentPTags:
    # Paragraphs (no regex because no anomalies)
    contentParagraphs.append(pTag.text)
    # Sentences: https://stackoverflow.com/questions/25735644/python-regex-for-splitting-text-into-sentences-sentence-tokenizing
    pat1 = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', pTag.text)
    contentSentences.extend(pat1)
    # Numbers: https://stackoverflow.com/questions/16321007/python-regular-expression-regex-match-comma-separated-number-why-does-this-n
    pat2 = re.compile(r'\d+(?:,\d+)?', re.M)
    contentNumbers.extend(pat2.findall(pTag.text))
    # Words counts: https://stackoverflow.com/questions/32324405/i-need-a-regular-expression-that-will-deal-with-apostrophes-when-extracting-word
    wordList = []
    pat3 = re.compile(r"(?:^|(?<=\s))[a-z0-9_'\-]+(?=\s|$|\b)", re.M)
    wordList.extend(pat3.findall(pTag.text.lower()))
    for word in wordList:
        if word not in contentWords:
            contentWords[word] = 1
        else:
            contentWords[word] += 1
    
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
contentSentencesDF = pandas.DataFrame({'sentence-text' : contentSentences})
contentNumbersDF = pandas.DataFrame({'numbers-text' : contentNumbers})
contentWordsDF = pandas.DataFrame.from_dict(contentWords, orient='index')
contentWordsDF.reset_index(inplace = True)
contentWordsDF.columns = ['Word','Count']

## <span style="color:blue">Notes on Section 1</span>
<span style="color:blue">
I make a request to wikisources for President Obama's last state of the union speech. From the beautiful soup object of the text returned from this request, I separate out the paragraphs designated by a 'p' tag. From each of these paragraphs, I make a few pandas dataframes. The first dataframe contains one paragraph in each row (no regex necessary, as the text was already clean). The second dataframe contains each of the sentences (the regex splits on positive lookbehinds and lookaheads using uppercase letters in the beginning and punctuation at the end). The third dataframe contains all the numbers in the text (the regex is able to capture commas in the numbers). The fourth dataframe shows a count of the number of times each word appeared in the text. I first regex out each word (apostrophes included, uppercase converted to lower case) and create a dictionary of each word and increment the count if the word is encountered again.</span>

In [10]:
display(contentParagraphsDF.head())
display(contentSentencesDF.head())
display(contentNumbersDF.head())
display(contentWordsDF.head())

,paragraph-text
0,"Mr. Speaker, Mr. Vice President, Members of Co..."
1,Tonight marks the eighth year that I’ve come h...
2,Now I understand that because it’s an election...
3,"But tonight, I want to go easy on the traditio..."
4,"But for my final address to this chamber, I do..."


,sentence-text
0,"Mr. Speaker, Mr. Vice President, Members of Co..."
1,Tonight marks the eighth year that I’ve come h...
2,"And for this final one, I’m going to try to ma..."
3,I know some of you are antsy to get back to Iowa.
4,I've been there.


,numbers-text
0,14
1,90
2,"900,000"
3,30
4,200


,Word,Count
0,mr,3
1,speaker,3
2,vice,2
3,president,6
4,members,1



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [167]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 1, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 2, 'texts'), ('https://en.wikipedia.org/wiki/Semantics', 2, 'meaningful'), ('https://en.wikipedia.org/wiki/Machine_learning', 3, 'Machine learning'), ('https://en.wikipedia.org/wiki/Klaus_Krippendorff', 6, 'Klaus Krippendorff'), ('https://en.wikipedia.org/wiki/Radio', 7, 'radio'), ('https://en.wikipedia.org/wiki/Television', 7, 'television'), ('https://en.wikipedia.org/wiki/Key_Word_in_Context', 7, 'Keyword In Context'), ('https://en.wikipedia.org/wiki/Synonym', 7, 'synonyms'), ('https://en.wikipedia.org/wiki/Homonym', 7, 'homonyms')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [32]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF.head()

,paragraph-text,source,paragraph-number
0,,https://en.wikipedia.org/wiki/Content_analysis,0
1,\nContent analysis is a research method for st...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,3
4,,https://en.wikipedia.org/wiki/Content_analysis,4


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [33]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [37]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF.head()

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,,https://en.wikipedia.org/wiki/Content_analysis,NaN,None
1,1,\nContent analysis is a research method for st...,https://en.wikipedia.org/wiki/Content_analysis,NaN,None
2,2,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,NaN,None
3,3,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,NaN,None
4,4,,https://en.wikipedia.org/wiki/Content_analysis,NaN,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [165]:
speech_urls = []
links_URL = 'https://millercenter.org/the-presidency/presidential-speeches'
prefix_URL = 'https://millercenter.org/'
ajax_URL = 'https://millercenter.org/views/ajax?_wrapper_format=drupal_ajax'

#payload and headers for the simulated ajax post request
ajax_POST_data = {'view_name':'presidential_speech',
                 'view_display_id':'presidential_speech_view_block',
                 'view_args':'all',
                 'view_path':'/the-presidency/presidential-speeches',
                 'view_base_path':'',
                 'view_dom_id':'84950cef194ed5113fc364b1d099c03e19a535fe57e7765d4dbd77cc5ea9915d',
                 'pager_element':'0',
                 'page':'0',
                 '_drupal_ajax':'1'}
ajax_POST_headers = {'Host': 'millercenter.org',      
                    'Referer': 'https://millercenter.org/the-presidency/presidential-speeches',
                    'X-Requested-With': 'XMLHttpRequest',
                    'Content-Length': '869'}

#Initializing session for simulating infinite scrolling ajax request
session = requests.Session()
response = session.post(url=ajax_URL, data=ajax_POST_data, headers=ajax_POST_headers)
while(response.status_code == 200):
    #hard coding to the response's expected structure
    ajaxhtmladdition = response.json()[-1]['data']
    links_soup = bs4.BeautifulSoup(ajaxhtmladdition, "lxml")
    tags_found = links_soup.find_all('a', href=True)
    for tag in tags_found:
        link = tag.attrs['href']
        if 'ajax' not in link: #it's a valid presidential speech link, not a drupal thing
            speech_urls.append(link)
    if len(tags_found) == 0:
        break
    ajax_POST_data['page'] = str(int(ajax_POST_data['page']) + 1)
    response = session.post(url=ajax_URL, data=ajax_POST_data, headers=ajax_POST_headers)


In [216]:
def getTextFromMillerCenter(baseUrl, urlList):
    
    #Dictionary to build for dataframe
    parsDict = {'source': [], 'text': [], 'title': [], 'president': []}

    for relurl in urlList:
        #Get page
        url = urllib.parse.urljoin(baseUrl, relurl)
        request = requests.get(url)
        contentSoup = bs4.BeautifulSoup(request.text, 'html.parser')
        
        #URL
        parsDict['source'].append(url)
        
        #Text of speech
        contentTag = contentSoup.findAll('div',{'class' : 'transcript-inner'})
        for content in contentTag:
            text0 = content.text[11:] #leave out first word ('transcript')
            text1 = re.sub(r'\n', ' ', text0)
            text2 = re.sub(r'\s\s', ' ', text1)  
            text = re.sub(r'\xa0', '', text2)  
            parsDict['text'].append(text.replace("(Applause.) ", ''))
        
        #President
        presidentTag = contentSoup.findAll('p',{'class' : 'president-name'})
        for president in presidentTag:
            parsDict['president'].append(president.text)
       
    
        #Title (date, occasion) of speech
        titleTag = contentSoup.findAll('title')
        for title in titleTag:
            parsDict['title'].append(title.text.replace(' | Miller Center', '')) 
    
    return pandas.DataFrame(parsDict)


In [217]:
getTextFromMillerCenter('https://millercenter.org/', speech_urls[:50])

,president,source,text,title
0,Donald Trump,https://millercenter.org/the-presidency/presid...,"Mr. Speaker, Mr. Vice President, Members of C...","February 28, 2017: Address to Joint Session of..."
1,Donald Trump,https://millercenter.org/the-presidency/presid...,"Chief Justice Roberts, President Carter, Pres...","January 20, 2017: Inaugural Address"
2,Barack Obama,https://millercenter.org/the-presidency/presid...,Hello Rutgers! R-U rah-rah! Thank you so much...,"May 15, 2016: Commencement Address at Rutgers ..."
3,Barack Obama,https://millercenter.org/the-presidency/presid...,Thank you. Muchas gracias. Thank you so much....,"March 22, 2016: Remarks to the People of Cuba"
4,Barack Obama,https://millercenter.org/the-presidency/presid...,"Mr. Speaker, Mr. Vice President, Members of C...","January 12, 2016: 2016 State of the Union Address"
5,Barack Obama,https://millercenter.org/the-presidency/presid...,THE PRESIDENT: Giving all praise and honor to...,"June 26, 2015: Remarks in Eulogy for the Honor..."
6,Barack Obama,https://millercenter.org/the-presidency/presid...,"UDIENCE MEMBER: We love you, President Obama!...","March 7, 2015: Remarks at the 50th Anniversary..."
7,Barack Obama,https://millercenter.org/the-presidency/presid...,"Mr. Speaker, Mr. Vice President, Members of C...","January 20, 2015: 2015 State of the Union Address"
8,Barack Obama,https://millercenter.org/the-presidency/presid...,"My fellow Americans, tonight, I’d like to tal...","November 20, 2014: Address to the Nation on Im..."
9,Barack Obama,https://millercenter.org/the-presidency/presid...,"Mr. Speaker, Mr. Vice President, Members of C...","January 28, 2014: 2014 State of the Union Address"


## <span style="color:blue">Notes on Section 2</span>
<span style="color:blue">
The first part of the code creates a list of url paths to US presidents' speeches on the Miller Center's website. The code uses ajax to go through the page containing all the links to all the presidential speeches from the present to the past and collects the links ('href' tags) to each of them. Then, the function getTextFromMillerCenter is specific to scraping text from each of these page links. Given the base url and a list of specific url paths (previously created), the function crawls through each of the webpages and filters out and records the president's name, the source (full url), text of the speech (cleaned through regex and replacements), and title of the speech (date and occasion).</span>

## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [46]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":2964,"posts-type":false,"posts":[{"id":"169306612050","url":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050","url-with-slug":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050\/kitty-mind-blown","type":"photo","date-gmt":"2018-01-04 15:00:13 GMT","date":"Thu, 04 Jan 2018 16:00:13","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1515078013,"format":"html","reblog-key":"TZzKhxon","slug":"kitty-mind-blown","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"169306612050\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_169306612050\"><iframe id=\"like_iframe_169306612050\" src=\"http:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=fc298e85f978b8662a643fe0a6b8c638#name=lolcats-lol-cat&amp;post_id=169306612050&amp;co

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [47]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [43]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)


In [44]:
lolcat = tumblrImageScrape('lolcats-lol-cat', 50)
lolcat.head()

,date,id,photo-type,photo-url,tags
0,"Thu, 04 Jan 2018 16:00:13",169306612050,jpg,http://78.media.tumblr.com/704a681944a94a1fac1...,"[cat, cats, lol, lolcat, lolcats]"
1,"Thu, 04 Jan 2018 14:00:41",169303827051,png,http://78.media.tumblr.com/1ec2cc33b059d657351...,"[cat, cats, lol, lolcat, lolcats]"
2,"Thu, 04 Jan 2018 12:00:34",169301536211,png,http://78.media.tumblr.com/80dc7c42c35a2dbce58...,"[cat, cats, lol, lolcat, lolcats]"
3,"Tue, 02 Jan 2018 04:00:09",169209300706,jpg,http://78.media.tumblr.com/94b8dbe91683bd01551...,"[cat, cats, lol, lolcat, lolcats]"
4,"Tue, 02 Jan 2018 02:00:42",169205268941,jpg,http://78.media.tumblr.com/e3bbb26992ac10e0234...,"[cat, cats, lol, lolcat, lolcats]"


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [51]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [52]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

In [53]:
with open('../data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [56]:
targetDir = 'C:/Users/Sun-joo Lee/Git_Projects/content-analysis-2018/data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, 'rt', encoding='UTF8') as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)


Then we can put them all in pandas DataFrame

In [58]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df.head()

,text
alls_well_that_ends_well.txt,"All's Well, that Ends Well\n\nActus primus. Sc..."
anthonie_and_cleopatra.txt,"The Tragedie of Anthonie, and Cleopatra\n\nAct..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
comedy_of_errors.txt,"DRAMATIS PERSONAE\n\nSOLINUS, Duke of Ephesus\..."
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [59]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [60]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [61]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [62]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [63]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [64]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [65]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [66]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [76]:
infoExtractionRequest = requests.get('http://rfkhumanrights.org/media/filer_public/af/a1/afa1ea83-1d15-47d7-aea9-37658772f554/nelson_mandela.pdf', stream=True)
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

In [130]:
mandelaSpeechText0 = readPDF(infoExtractionBytes)[1187:6549]
mandelaSpeechText1 = re.sub(r'\n', ' ', mandelaSpeechText0)
mandelaSpeechText2 = re.sub(r'\s\s', ' ', mandelaSpeechText1)
mandelaSpeechText = re.sub(r'\x0c', '', mandelaSpeechText2)  

In [156]:
title = mandelaSpeechText[:192]
text = mandelaSpeechText[193:]
mandelaSpeechDict = {'speech-title' : title, 'speech-text' : text}

In [160]:
speeches = []
speeches.append(mandelaSpeechDict)

SpeechDF = pandas.DataFrame(speeches)
SpeechDF = mandelaSpeechDF[['speech-title', 'speech-text']]
SpeechDF

,speech-title,speech-text
0,STATEMENT OF THE PRESIDENT OF THE AFRICAN NATI...,"Your Majesties, Your Highnesses, Distinguished..."


## <span style="color:blue">Notes on Section 3</span>
<span style="color:blue">
    I make a request for the pdf of the speech of Nelson Mandela's inaugural address. Then I extract the bytes and read in the text of the pdf using the code provided. The pdf has a lot of explanatory text before and after the speech, so I select only the actual speech from the text through indexing. I clean up the text with a few regex substitutions, then split the speech into a the title and the text and create a dictionary. I create a dataframe showing the title and text of the speech from the dictionary. I can add to this pandas dataframe by creating more such SpeechDicts and appending them to the speeches list. </span>